In [1]:
import pandas as pd
import numpy as np
import cloudpickle
import pickle
import time
import dask.dataframe as dskdf
import dask
gl = globals()

In [ ]:
class test:

    def __init__(self):
        pass

    #地域系は全部で４つのデータをマージする
    #車種系は全部で●つのデーたをマージする
    #車種の元ネタが成約データなので、最終的に車種データに地域データを市区町村名で紐付ける

    #地域(1)
    #市区町村別国勢調査
    def make_kokusei(self):
        kokusei = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/kokuseichosa_2015_for_merge.csv")
        #全てを英字変換する必要あり
        kokusei.columns=["pref_code", "pref_city_code", "capital_city_flg", "city_categories", "chiho", "city_name",
                         "population", "h22_kumikae_population", "population_changed_from_h22_to_h27", "rate_population_changed_from_h22_to_h27",
                         "gross_area", "population_density", "ave_age_from_h22_to_h27", "med_age_from_h22_to_h27", "population",
                         "population_under_15old", "population_from_15_to_64_old", "population_over_65old", "rate_population_under_15_year_old",
                         "rate_population_from_15_to_64_old", "rate_population_over_65old", "population_male","population_male_under_15old",
                         "population_male_from_15_to_64_old", "population_male_over_65old", "rate_population_male_under_15old",
                         "rate_population_male_from_15_to_64_old", "rate_population_male_over_65old", "population_female",
                         "population_female_under_15old", "population_female_from_15_to_64_old", "population_female_over_65old",
                         "rate_population_female_under_15old", "rate_population_female_from_15_to_64_old", "rate_population_female_over_65old",
                         "rate_population_male_to female", "population_jap", "population_no_jap", "num_of_family", "num_of_normal_family",
                         "num_of_family_in_facilities", "h22_kumikae_family", "num_of_normal_family", "num_of_nuclear_family",
                         "num_of_family_no_children", "num_of_family_with_children", "num_of_father_and_children", "num_of_mother_and_children",
                         "num_of_singles", "num_of_over_65old_singles", "num_of_old_married_couples_no_children", "num_of_3generation_family",]
        return kokusei

    #chiho(都道府県名)で国勢調査と保有台数をマージ
    def left_merge_on_chiho(self,left,right,out):
        out = pd.merge(left,right,on="chiho",how="left")
        #一番下に謎の欠損行が存在するので削除する
        out = out.dropna(subset=['pref_code'])
        return out

    #地域(2)
    #都道府県別保有台数
    def make_hoyu(self):
        hoyu = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/2015_hoyu_daisu.csv",encoding="utf-8")
        hoyu.columns = ["chiho", "num_of_cars_all", "num_of_cars_kei", "num_of_cars_normal", "num_of_familiy", "cars_per_person",
                        "normal_cars_per_person", "kei_cars_per_person", "kei_rate", "normal_rate", "kei_share_in_JP", "normal_share_in_JP",
                        "guess_2nd_hand_kei_car_sales_per_year", "guess_2nd_hand_normal_car_sales_per_year", "guess_2nd_hand_car_all_sales_per_year"]
        koku_hoyu = left_merge_on_chiho(kokusei, hoyu, "koku_hoyu")
        return koku_hoyu

In [ ]:
tes = test()
tes.make_kokusei()
tes.make_hoyu()

tes.kokusei.head()
tes.koku_hoyu.head(6)


In [2]:
ku = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_1.csv",delimiter="\t",encoding="cp932")
ru = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_2.csv",delimiter="\t",encoding="cp932")
ma = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_3.csv",delimiter="\t",encoding="cp932")

/Users/01017387/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
kuru = pd.concat([ku,ru],axis=0)


In [4]:
print(ku.shape)
print(ru.shape)
print(kuru.shape)


(1378603, 30)
(1378534, 30)
(2757137, 60)


In [142]:
# kuruma = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/tes.csv")
ku = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_1.csv",
                 delimiter="\t", dtype="object", encoding="cp932",)
colname = ku.columns
ru = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_2.csv",
                 delimiter="\t", dtype="object", encoding="cp932",names=colname)
ma = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_3.csv",
                 delimiter="\t", dtype="object", encoding="cp932",names=colname)
kuru = pd.concat([ku,ru],axis=0)
kuruma = pd.concat([kuru,ma],axis=0)

kuruma["b.shikuchoson_jusho_kj"] = kuruma["b.shikuchoson_jusho_kj"].str.replace('ヶ', 'ケ')
kuruma["b.todofuken_jusho_kj"] = kuruma["b.todofuken_jusho_kj"].map(lambda x: str(x).strip())
#県単位、市区町村単位での地方別集計（問い合わせ）
#もともとの市区町村名に入ってるレベル感が違う(市区町村で終わるものもあれば、町大字まで入っているものもある)
#市区町村マスタと一致する部分のみ残す

#都道府県欠損を削除
cu = kuruma.dropna(subset=["b.todofuken_jusho_kj"])
#都道府県名に異常値が入っているものを除く
# '〓'が入っていたらアウト
cus = cu.ix[cu['b.shikuchoson_jusho_kj'] != "〓"]
cus = cus.ix[cus['b.shikuchoson_jusho_kj'] != "null"]


In [113]:
dl = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/市区町村マスタ.csv",names=["pref","city"])

#h市区町村表記揺れを消したい
dl["city"] = dl["city"].str.replace('ヶ', 'ケ')
pre = dl["pref"].unique() #array
#県内市区町村数のカウント用
dl["count"] =1
num = dl.groupby("pref").sum()
dc = num.to_dict()
dic = dc["count"]

biglis = []
c=0
#都道府県別市区町村の格納
for i in pre:
    tmplis = []
    for k in range(int(dic[i])):
#             exec("tmplis.append(dl['city']["+str(c)+"])")
        tmplis.append(dl['city'][c])
        c +=1
    #元データの入り方が札幌市→札幌市〇〇区、といった順で入っているので、上から部分一致で合致を探す際の障害になる
    #ゆえに最初の要素を最後に持ってくる処理が必要
    push_to_last = tmplis.pop(0)
    tmplis.append(push_to_last)
    biglis.append(tmplis)

#辞書の作成
#県名をキーとして、市区町村名のリストをバリューとする
dicts = {key: value for (key, value) in zip(pre,biglis)}


In [143]:

#dfの各要素について、辞書のどこに一致するかを見て、ループで部分一致を探す
#部分一致したものをtoshiで保持する
# ken = kuruma["b.todofuken_jusho_kj"].values
# ken = kuruma["b.todofuken_jusho_kj"].map(lambda x: str(x).strip()).values
# org_toshi = kuruma["b.shikuchoson_jusho_kj"].values
ken = cus["b.todofuken_jusho_kj"].map(lambda x: str(x).strip()).values
org_toshi = cus["b.shikuchoson_jusho_kj"].values
toshi = [] #整形した市区町村名
counter =[] #部分一致するものがあるかどうかのフラグ
cnt =0
for k,i in zip(ken,org_toshi):
#     print(dicts[k])
#     cnt +=1
#     if cnt>26304:
#         print(k)
#         print(dicts[k])
    count=0
    for l in dicts[k]:
        if l in i:
            toshi.append(l)
            counter.append(1)
            break
        count+=1
        if count >=(len(dicts[k])):
            toshi.append("")
            counter.append(0)
            break


In [145]:
#概ねOKだけど、県名の後に直接郡の名前が来るケースに対応できていない
# →修正面倒なので後回し
#スクレイピングで持ってくる必要ある：　http://www.geocities.jp/je2kcr/current_allgun.htm
#toshiが欠損のものはとりあえず除外検討

#bbをkurumaにマージ
cus["city_name"] = toshi
cus["count"] = 1
#toiawase_dateの型を日付型に変更する必要がある
cus["toiawase_datetime"] = pd.to_datetime(cus["b.toiawase_date"].str[:10] + " " + cus["b.toiawase_date"].str[10:])
cus["toiawase_y"] = cus["toiawase_datetime"].dt.year.astype(str)
cus["toiawase_m"] = cus["toiawase_datetime"].dt.month.astype(str)
cus["toiawase_ym"] = cus["toiawase_y"] + cus["toiawase_m"]


In [ ]:
import pandas as pd
import numpy as np
import cloudpickle
import pickle
import time
import dask.dataframe as dskdf
import dask

gl = globals()

class tes:
    # -*- coding: utf-8 -*-

    def make_catalogue(self):
        cat_list = ["mc1_body_type.txt", "mc1_kanren_shashu.txt", "mc1_brand.txt", "mc1_kihon_iro.txt",
                    "mc1_brand_country.txt",
                    "mc1_kukuri_kanren_shashu.txt", "mc1_brand_iro.txt", "mc1_maker.txt", "mc1_brand_maker.txt",
                    "mc1_mc_model.txt",
                    "mc1_country.txt", "mc1_mc_model_gazo.txt", "mc1_fmc_model.txt", "mc1_net_category.txt",
                    "mc1_fp_shashu_midashi_settei_work.txt", "mc1_shashu.txt", "mc1_grade.txt", "mc1_sobi.txt",
                    "mc1_grade_iro.txt",
                    "mc1_transmission_henkan.txt", "mc1_grade_sobi.txt"]
        index_name = ["ボディ形状マスタ", "関連車種マスタ", "ブランドマスタ", "基本色マスタ", "ブランド国設定マスタ", "括り関連車種マスタ",
                      "ブランド色マスタ", "メーカーマスタ", "ブランドメーカー設定マスタ", "MCモデルマスタ", "国マスタ", "MCモデル画像マスタ",
                      "FMCモデルマスタ", "NETカテゴリマスタ", "FP車種見出し設定マスタ", "車種マスタ", "グレードマスタ", "装備マスタ",
                      "グレード色設定マスタ", "トランスミッション変換マスタ", "グレード装備設定マスタ", ]
        df_columns = [["body_shape_cd", "body_shape_nm", "register_ymd", "last_modified_ymd"],
                      ["kanren_shashu_cate", "kanren_shashu_cate_no", "kanren_kj", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "brand_nm", "brand_hyoji_order", "catalogue_keisai_fuka_cate", "seo_folder_nm",
                       "register_ymd", "last_modified_ymd"],
                      ["base_color_cd", "base_color_nm", "base_color_hyoji_jun", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "country_cd", "daihyo_country_flg", "register_ymd", "last_modified_ymd"],
                      ["kanren_shashu_cate", "kanren_shashu_cate_no", "brand_cd", "shashu_cd",
                       "kukuri_kanren_shashu_hyoji_jun",
                       "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "brand_color_cd", "brand_color_nm", "base_color_cd", "jitsu_brand_color_nm",
                       "brand_color_hyoji_order", "base_color_cd1", "base_color_cd2", "register_ymd", "last_modified_ymd"],
                      ["maker_cd", "maker_nm", "maker_hyoji_jun", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "maker_cd", "daihyo_maker_flg", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "mc_hanbai_start", "mc_hanbai_end", "mc_cate", "mc_kj",
                       "mc_commment_title", "mc_comment", "catalogue_keisai_fuka_cate", "register_ymd",
                       "last_modified_ymd"],
                      ["country_cd", "country_nm", "area_cate", "country_hyoji_jun", "register_ymd", "daihyo_base_color",
                       "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "gazo_index_no", "mc_model_pic_hyoji_jun",
                       "gazo_cate_cd",
                       "caption", "catalogue_keisai_fuka_cate", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "fmc_hanbai_start", "fmc_hanbai_end", "fmc_kj",
                       "catalogue_keisai_fuka_cate", "register_ymd", "last_modified_ymd"],
                      ["net_cate_cd", "cate_nm", "net_cate_hyoji_jun", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "fp_cate_cd", "fp_shashu_midashi_cd",
                       "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "shashu_nm", "wareki_seireki_cate", "shashu_hyoji_jun",
                       "catalogue_keisai_fuka_cate", "search_key_1", "search_key_2", "search_key_3", "seo_folder_nm",
                       "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "grade_kukuri_cd", "grade_nm",
                       "grade_hyoji_jun",
                       "body_shape_cd", "grade_hanbai_start_ym", "grade_hanbai_end_ym", "katashiki", "ruibetsu_kigo",
                       "katashiki_shitei_no", "ruibetsu_cate_no", "price_with_tax", "price_without_tax",
                       "nenpi_kijun_achieve_cate", "nenpi_kijun_nen", "low_emission_nintei_stars",
                       "low_emission_nintei_nen",
                       "reduce_tax_cate", "heiko_yunyusha_cate", "walfare_car_cate", "car_for_commercial_use", "kei_flg",
                       "number_size", "special_edition_flg", "catalogue_keisai_fuka_cate", "keisai_commentary",
                       "kudo_hoshiki_cate", "transmission_su", "transmission_cate", "transmission_ichi_cate",
                       "transmission_cd",
                       "handle_ichi_cate", "length", "width", "height", "inner_length", "inner_width", "inner_height",
                       "wheel_base", "front_tred", "back_tred", "min_shako", "weight_of_car", "weight_of_car_all",
                       "max_carriage", "max_carriage_persons", "min_kaiten_hankei", "10_15_mode_nenpiritsu",
                       "stearing_gear_cate", "stearing_gear_nm", "front_suspension_cate", "front_suspension_nm",
                       "back_suspension_cate", "back_suspension_nm", "front_break_cate", "front_break_nm",
                       "back_break_cate",
                       "back_break_nm", "front_tire_txt", "back_tire_txt", "engine_katashiki", "engine_valve_cate",
                       "engine_silinder_cate", "engine_kito_su", "engine_kakyuki_cate", "engine_cate", "silinder_radius",
                       "silinder_koutei", "total_haikiryo", "haikiryo1", "haikiryo2", "asshukuhi", "max_shuturyoku_kw",
                       "max_shuturyoku_ps", "max_shuturyoku_rmp_max", "max_shuturyoku_rmp_min", "max_torque_mn",
                       "max_torque_kgm", "max_torque_rpm_max", "max_torque_rpm_min", "fuel_sapply_device_cate", "fuel_cate",
                       "fuel_tank_capacity", "door_su", "seat_columns", "net_cate_cd", "register_ymd", "last_modified_ymd",
                       "jc08_mode_nenpi_rtsu"],
                      ["equipment_cd", "equipment_nm", "equipment_bunrui_cate", "catalogue_keisai_cate",
                       "equipment_hyoji_jun",
                       "equipment_usage_cd", "register_ymd", "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "brand_color_cd", "option_flg",
                       "option_price",
                       "grade_hyoji_jun", "register_ymd", "last_modified_ymd"],
                      ["transmission_cd", "transmission_su", "transmission_cate", "transmission_ichi_cate", "register_ymd",
                       "last_modified_ymd"],
                      ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "equipment_cd", "equipment_nm",
                       "equipment_jokyo_cd", "nm", "register_ymd", "last_modified_ymd"]
                      ]
        # 読んだ後にとりあえず全部くっつけたい
        co = 0
        df_names = []
        for ct in cat_list:
            df_names.append(ct.rstrip(".txt")[4:])

        start = time.time
        self.dfs = []
        cnt = 0
        for cat, dn, dc in zip(cat_list, df_names, df_columns):
            var = 'cat_' + str(dn)  # 変数名
            tgt = '/Users/01017387/Desktop/しごと/data/car用/車カタログ/' + str(cat)

            if cat in ["mc1_grade_sobi.txt", "mc1_grade_iro.txt", "mc1_brand_iro.txt"]:
                gl[var] = pd.read_csv(tgt, encoding="cp932", delimiter="\t", names=dc, nrows=10000)
            # exec('cat_'+str(dn)+' = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/車カタログ/'+str(cat)+'",encoding="cp932",delimiter="\t",names='+str(dc)+')')
            # 型指定がうまくいかなくてエラーになるので、すべてobjectで読み込む
            else:
                gl[var] = dskdf.read_csv(tgt, encoding="cp932", delimiter="\t", dtype="object", names=dc).head(n=10000)
            # exec('cat_'+str(dn)+' = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/車カタログ/'+str(cat)+'",encoding="cp932",delimiter="\t",low_memory=False,dtype="object",names='+str(dc)+')')
            cnt += 1
            # 登録日、更新日は不要なので読み込み時点で落とす
            #     exec('"cat_"+'+str(dn)+'["last_modified_tmd"]')

            # データ名もリストに格納しておく
            self.dfs.append("self.cat_" + str(dn))
        # print('cat_'+str(dn))
        #     print(cat)
        #     co +=1
        #     if co ==17:
        #         break

        # 少しでも軽くするために、不要な変数を削除しておく
        for dd in self.dfs:
            # print(dd)
            gl[dd] = gl[dd].drop("register_ymd", axis=1)
            gl[dd] = gl[dd].drop("last_modified_ymd", axis=1)
            if dd == "cat_grade":
                gl[dd] = gl[dd].drop("number_size", axis=1)
                gl[dd] = gl[dd].drop("catalogue_keisai_fuka_cate", axis=1)
            if dd == "cat_mc_model":
                gl[dd] = gl[dd].drop("catalogue_keisai_fuka_cate", axis=1)
            if dd == "cat_shashu":
                gl[dd] = gl[dd].drop("catalogue_keisai_fuka_cate", axis=1)
                gl[dd] = gl[dd].drop("search_key_1", axis=1)
                gl[dd] = gl[dd].drop("search_key_2", axis=1)
                gl[dd] = gl[dd].drop("search_key_3", axis=1)
                gl[dd] = gl[dd].drop("seo_folder_nm", axis=1)
            if dd == "cat_sobi":
                gl[dd] = gl[dd].drop("equipment_hyoji_jun", axis=1)

        # 作成したデータは全て後ほど使うので、returnで返しておく
        return self.dfs
    # カタログ全体のマージ準備

    def catalogue_merge_pre(self):
        # 装備ごとに作られているテーブルが存在するので、装備をダミー変数化するためにpivotする必要がある
        # ダミー化した変数の内容がわかるように、全てにsobi_の接頭辞をつけておく
        self.cat_grade_sobi_pv = pd.pivot_table(self.cat_grade_sobi,
                                                index=["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd"],
                                                columns="equipment_nm", values="equipment_jokyo_cd").reset_index()
        col_equip = ["sobi_" + str(x) for x in self.cat_grade_sobi_pv.columns[5:, ]]
        bs = ['brand_cd', 'shashu_cd', 'fmc_cd', 'mc_cd', 'grade_cd']
        bs.extend(col_equip)
        self.cat_grade_sobi_pv.columns = bs

        # grade_iroは先にbrand_iroとマージして、基本色を獲得しておく
        cat_grade_iro["count"] = 1
        cat_grade_brand_iro = pd.merge(cat_grade_iro, cat_brand_iro.ix[:, ["brand_cd", "brand_color_cd", "base_color_cd"]],
                                       on=["brand_cd", "brand_color_cd"])
        self.sum_grade_brand_iro = pd.pivot_table(cat_grade_brand_iro,
                                                  index=["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd"],
                                                  columns="base_color_cd", values="count").reset_index()

        # ダミー化した変数の内容がわかるように、全てにbase_color_の接頭辞をつけておく
        col_colors = ["base_color_" + str(x) for x in self.sum_grade_brand_iro.columns[5:, ]]
        bs = ['brand_cd', 'shashu_cd', 'fmc_cd', 'mc_cd', 'grade_cd']
        bs.extend(col_colors)
        self.sum_grade_brand_iro.columns = bs

        return [self.cat_grade_sobi_pv, self.sum_grade_brand_iro]


    # カタログ全体のマージ
    def catalogue_merge(self):
        # 上記前処理が終わったら、ループで各種データをマージする
        merge_key = [["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd"], ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd"],
                     ["brand_cd", "shashu_cd", "fmc_cd"], ["brand_cd", "shashu_cd"], ["brand_cd", "shashu_cd"],
                     ["body_shape_cd"], ["transmission_cd"], ["brand_cd"], ["brand_cd"], ["brand_cd"],
                     ["kanren_shashu_cate"]
                     ]

        merge_data = ["self.cat_grade", "self.cat_mc_model", "self.cat_fmc_model", "self.cat_shashu", "self.cat_kukuri_kanren_shashu",
                      "self.cat_body_type",
                      "self.cat_transmission_henkan", "self.cat_brand_country", "self.cat_brand_maker", "self.cat_brand", "self.cat_kanren_shashu"
                      ]

        loops = np.array(list(range(11))) + 2

        pandas_list = ["mc1_body_type.txt", "mc1_kanren_shashu.txt"]

        # dask_ver
        # a1 = cat_grade_sobi.merge(cat_grade_iro,on=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"])
        # pandas_ver
        moto = pd.merge(self.cat_grade_sobi_pv, self.sum_grade_brand_iro,
                        on=["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd"])
        # a1をdask化
        a1 = dskdf.from_pandas(moto, npartitions=1)

        cnt = 0
        # dask_ver
        print(loops)
        for mk, md, lp in zip(merge_key, merge_data, loops):
            #     a2 = a1.merge(cat_grade,on=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"])
            var = 'a' + str(lp)
            var_1 = 'a' + str(lp - 1)

            print(var)
            print(var_1)
            gl[var] = gl[var_1].merge(gl[md], on=mk)

        self.a12 = a12.copy()
        # ファイルが死ぬほど大きいので退避させておく
        self.a12.to_csv("/Users/01017387/Desktop/しごと/data/car用/車カタログ/test/car_catalogue_ss.csv")
        return self.a12

In [ ]:

te = tes()
te.make_catalogue()
te.catalogue_merge_pre()
te.catalogue_merge()

